# Coherent info

In [1]:
import numpy as np
from collections import Counter
from typing import Tuple, Iterable, Dict, List
from coherentinfo.moebius import MoebiusCode
from coherentinfo.linalg import finite_field_gauss_jordan_elimination

In [2]:
length = 7
width = 5
moebius_code = MoebiusCode(length=length, width=width, d=2)
h_z = moebius_code.h_z
h_x = moebius_code.h_x
logical_x = moebius_code.logical_x
logical_z = moebius_code.logical_z

In [3]:
vertex_destab = moebius_code.vertex_destab

In [4]:
print(vertex_destab)

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ... -1  0  0]
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0 -1]]


In [5]:
id_mat = np.identity(moebius_code.num_vertex_checks)
print(np.count_nonzero(h_z @ vertex_destab.T - id_mat))

0


In [6]:
moebius_code.num_vertex_checks

28

In [7]:
print(logical_z @ vertex_destab.T)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
h_x_qubit = moebius_code.h_x_qubit
plaquette_destab_qubit = moebius_code.plaquette_destab_qubit

In [41]:
index = 34
h_x_qubit[index, :] @ plaquette_destab_qubit.T % 2

IndexError: index 34 is out of bounds for axis 0 with size 34

In [10]:
moebius_code.index_h(0, 1)

1

In [11]:
logical_x @ plaquette_destab_qubit.T % 2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int8)

In [15]:
moebius_code.length

7